In [1]:
import numpy as np
import tensorflow as tf

In [2]:
n_links = 5
n_nodes = 4

adjacencies = [
	[0, 1],
	[0, 2],
	[1, 2],
	[1, 3],
	[2, 3]
]

r, s = 0, 3
d_rs = 6.0

In [15]:
m_true = np.array([10., 1., 1., 1., 10.])
m = tf.Variable(np.array([9.0, 2.0, 2.0, 2.0, 9.0]), name="m", trainable=True, dtype=tf.float64)
b_true = np.array([0., 50., 10., 50., 0.])
b = tf.Variable(b_true, name="b", trainable=False, dtype=tf.float64)
x = tf.constant(np.array([4., 2., 2., 2., 4.]), dtype=tf.float64)
#true_cost = np.array([40, 52, 12, 52, 40], dtype=np.float64)
true_cost = m_true * x + b_true

optimizer = tf.keras.optimizers.Adam()

In [16]:
class StochasticNetworkLoading(tf.keras.Model):
	def __init__(self, m, b, d, r, s, adjacencies, n_nodes, *args, **kwargs):
		super().__init__(*args, **kwargs)
		self.m = m
		self.b = b
		self.d = d
		self.adjacencies = adjacencies
		self.n_nodes = n_nodes
		self.r = r
		self.s = s
		self.I = np.identity(self.n_nodes)

	def link_cost(self, x):
		return tf.multiply(self.m, x) + self.b

	def link_logits(self, x):
		return tf.exp(-self.link_cost(x))

	def transition_logits(self, x):
		return tf.sparse.SparseTensor(
			indices=self.adjacencies,
			values=self.link_logits(x),
			dense_shape=(self.n_nodes, self.n_nodes)
		)

	def normalizer(self, L):
		V = tf.linalg.inv(self.I - tf.sparse.to_dense(L))
		V_rs = V[self.r, self.s]
		V_r = V[self.r, :]
		V_s = V[:, self.s]
		return tf.tensordot(V_r, V_s, axes=0) / V_rs

	def link_probabilities(self, x):
		L = self.transition_logits(x)
		W = self.normalizer(L)
		H = L * W
		return H.values

	def call(self, x):
		x = tf.cast(x, tf.float64)
		p = self.link_probabilities(x)
		return self.d * p

In [17]:
model = StochasticNetworkLoading(m, b, d_rs, r, s, adjacencies, n_nodes)
model.trainable_variables

[<tf.Variable 'm:0' shape=(5,) dtype=float64, numpy=array([9., 2., 2., 2., 9.])>]

In [19]:
for i in range(1000):
	with tf.GradientTape() as tape:
		x_hat = model(x)
		loss = (
			tf.reduce_mean((x - x_hat)**2)
		)
	grads = tape.gradient(loss, model.trainable_variables)
	optimizer.apply_gradients(zip(grads, model.trainable_variables))
	if i % 100 == 0:
		print(i, loss.numpy(), x_hat.numpy())

0 4.9307462025315374e-08 [4.00017555 1.99982445 2.0003511  1.99982445 4.00017555]
100 1.7351900848237788e-09 [4.00003293 1.99996707 2.00006586 1.99996707 4.00003293]
200 4.401089003982941e-11 [4.00000524 1.99999476 2.00001049 1.99999476 4.00000524]
300 7.839510323892962e-13 [4.0000007 1.9999993 2.0000014 1.9999993 4.0000007]
400 9.502714168668681e-15 [4.00000008 1.99999992 2.00000015 1.99999992 4.00000008]
500 7.538301513354946e-17 [4.00000001 1.99999999 2.00000001 1.99999999 4.00000001]
600 3.724202547475347e-19 [4. 2. 2. 2. 4.]
700 1.073236552528793e-21 [4. 2. 2. 2. 4.]
800 1.6356528365361054e-24 [4. 2. 2. 2. 4.]
900 5.164879422469585e-27 [4. 2. 2. 2. 4.]


In [20]:
print(f"Learned cost = {model.link_cost(x).numpy()}")
print(f"m = {m.numpy()}")
print(f"b = {b.numpy()}")

Learned cost = [38.00000008 53.00000012 15.00000004 53.00000012 38.00000008]
m = [9.50000002 1.50000006 2.50000002 1.50000006 9.50000002]
b = [ 0. 50. 10. 50.  0.]


In [10]:
true_model = StochasticNetworkLoading(m_true, b_true, d_rs, r, s, adjacencies, n_nodes)

assert np.allclose(true_model.link_cost(x).numpy(), true_cost)
assert np.allclose(true_model(x), x)
assert np.allclose(true_model.link_probabilities(x),
				   model.link_probabilities(x))

model.link_probabilities(x).numpy()

array([0.66666667, 0.33333333, 0.33333333, 0.33333333, 0.66666667])